In [3]:
pip install selenium webdriver-manager bs4

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup, UnicodeDammit
import csv
import time
from urllib.parse import urljoin
import os
import pandas as pd

BASE_DOMAIN = "https://digishop.vnpt.vn"
BASE_URL = f"{BASE_DOMAIN}/tin-tuc-km/tat-ca"

OUTPUT_PATH = r"C:\Users\Admin\Documents\VIETTEL\2-crawl-articles\scraper\output-scraper"
OUTPUT_FILE = os.path.join(OUTPUT_PATH, "vnpt_strategy.csv")

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(BASE_URL)

# wait for JS to load (increase if your connection is slow)
time.sleep(4)

# Get the full HTML from browser (more reliable than .page_source in some cases)
raw_html = driver.execute_script("return document.documentElement.outerHTML;")

# Force correct unicode detection
dammit = UnicodeDammit(raw_html)
html = dammit.unicode_markup

soup = BeautifulSoup(html, "html.parser")
driver.quit()

news_items = soup.find_all("div", class_="box-news")

records = []
for item in news_items:
    #Source
    source = "VNPT Strategy"

    # Title
    h3 = item.find("h3")
    title = h3.get_text(strip=True) if h3 else ""

    # URL (make absolute)
    url = ""
    if h3:
        a = h3.find("a")
        if a and a.has_attr("href"):
            url = urljoin(BASE_DOMAIN, a["href"])

    # Date
    date_tag = item.find("p", class_="box-news__datetime")
    date = date_tag.get_text(strip=True) if date_tag else ""

    # Description
    desc_tag = item.find("div", class_="box-news__description")
    description = desc_tag.get_text(strip=True) if desc_tag else ""

    records.append([source, title, url, date, description])

# Save CSV
df = pd.DataFrame(records, columns=["Nguồn", "Tiêu đề", "Mô tả", "Ngày", "URL"])
output_xlsx = os.path.join(OUTPUT_PATH, "vnpt_news.xlsx")
df.to_excel(output_xlsx, index=False)

print(f"Done! Saved {len(records)} items → {output_xlsx}")

✔ Done! Saved vnpt_news_utf8_bom.csv (10 items)
